Start

In [36]:
import numpy as np
import pandas as pd

# Pre-processing
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import cv2
from tensorflow.keras.utils import to_categorical
# Modeling
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional, Conv2D, MaxPooling2D

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/timkuz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
data = pd.read_csv('histology_slides/breast/image_data.csv')

In [38]:
data.head()

,path_to_image,Benign or Malignant,Cancer Type,Magnification
0,BreaKHis_v1/histology_slides/breast/benign/SOB...,Benign,Adenosis,100X
1,BreaKHis_v1/histology_slides/breast/benign/SOB...,Benign,Adenosis,100X
2,BreaKHis_v1/histology_slides/breast/benign/SOB...,Benign,Adenosis,100X
3,BreaKHis_v1/histology_slides/breast/benign/SOB...,Benign,Adenosis,100X
4,BreaKHis_v1/histology_slides/breast/benign/SOB...,Benign,Adenosis,100X


In [39]:
data['path_to_image'][0]

'BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis/SOB_B_A_14-22549AB/100X/SOB_B_A-14-22549AB-100-011.png'

In [40]:
def load_and_preprocess_images(image_paths, target_size=(224, 224)):
    images = []
    for path in image_paths:
        # Load image
        image = cv2.imread(path)
        # Resize image to target size
        image = cv2.resize(image, target_size)
        # Normalize pixel values to [0, 1]
        image = image / 255.0
        images.append(image)
    return np.array(images)

In [41]:
# Binary labels for Stage 1
data['Benign/Malignant'] = data['Benign or Malignant'].map({'Benign': 0, 'Malignant': 1})

data = data.drop(columns = ['Benign or Malignant'])


In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7909 entries, 0 to 7908
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   path_to_image     7909 non-null   object 
 1   Cancer Type       7905 non-null   object 
 2   Magnification     7905 non-null   object 
 3   Benign/Malignant  7906 non-null   float64
dtypes: float64(1), object(3)
memory usage: 247.3+ KB


In [43]:
data.head()

,path_to_image,Cancer Type,Magnification,Benign/Malignant
0,BreaKHis_v1/histology_slides/breast/benign/SOB...,Adenosis,100X,0.0
1,BreaKHis_v1/histology_slides/breast/benign/SOB...,Adenosis,100X,0.0
2,BreaKHis_v1/histology_slides/breast/benign/SOB...,Adenosis,100X,0.0
3,BreaKHis_v1/histology_slides/breast/benign/SOB...,Adenosis,100X,0.0
4,BreaKHis_v1/histology_slides/breast/benign/SOB...,Adenosis,100X,0.0


In [44]:
# Extract image paths and labels
image_paths = data['path_to_image']
ben_mal_labels = data['Benign/Malignant']
cancer_types = data['Cancer Type']

# Preprocessing images
images = load_and_preprocess_images(image_paths)

[ WARN:0@81637.826] global loadsave.cpp:241 findDecoder imread_('BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis/SOB_B_A_14-22549AB/100X/SOB_B_A-14-22549AB-100-011.png'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
# Split for binary classification
X_train_binary, X_val_binary, y_train_binary, y_val_binary = train_test_split(
    images, ben_mal_labels, test_size=0.2, random_state=42, stratify=ben_mal_labels)

In [ ]:
# Split for multiclass classification
X_train_multiclass, X_val_multiclass, y_train_multiclass, y_val_multiclass = train_test_split(
    images, cancer_types, test_size=0.2, random_state=42, stratify=cancer_types)

# Convert labels to categorical
y_train_multiclass = to_categorical(y_train_multiclass, num_classes=7)
y_val_multiclass = to_categorical(y_val_multiclass, num_classes=7)